In [ ]:
import os
import json
from PIL import Image

# Define YOLO dataset base path and COCO output path
yolo_base = '/path/to/yolo/dataset'
coco_output = '/path/to/coco/output'

# Define categories (update the names accordingly)
categories = [
    {"id": 0, "name": "delamination"},
    {"id": 1, "name": "crack"}
]

# Convert a YOLO split (train/val/test) to COCO format
def convert_split(split_name):
    image_dir = os.path.join(yolo_base, split_name, 'images')
    label_dir = os.path.join(yolo_base, split_name, 'labels')

    coco = {
        "info": {},
        "licenses": [],
        "categories": categories,
        "images": [],
        "annotations": []
    }

    ann_id = 0
    for img_id, image_file in enumerate(os.listdir(image_dir)):
        if not image_file.lower().endswith(('.jpg', '.png', '.jpeg')):
            continue
        
        img_path = os.path.join(image_dir, image_file)
        label_path = os.path.join(label_dir, os.path.splitext(image_file)[0] + ".txt")
        
        # Read image
        image = Image.open(img_path)
        width, height = image.size
        
        # Add image metadata
        coco["images"].append({
            "id": img_id,
            "file_name": image_file,
            "width": width,
            "height": height
        })

        if not os.path.exists(label_path):
            continue

        # Read corresponding label
        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) != 5:
                    continue
                class_id, x_center, y_center, w, h = map(float, parts)

                x = (x_center - w / 2) * width
                y = (y_center - h / 2) * height
                bbox_width = w * width
                bbox_height = h * height

                coco["annotations"].append({
                    "id": ann_id,
                    "image_id": img_id,
                    "category_id": int(class_id),
                    "bbox": [x, y, bbox_width, bbox_height],
                    "area": bbox_width * bbox_height,
                    "iscrowd": 0
                })
                ann_id += 1

    # Save to JSON
    os.makedirs(os.path.join(coco_output, 'annotations'), exist_ok=True)
    with open(os.path.join(coco_output, 'annotations', f'{split_name}.json'), 'w') as f:
        json.dump(coco, f, indent=2)

# Convert each split
for split in ['train', 'val', 'test']:
    convert_split(split)


In [1]:
import os
import re

# Directories for images and labels
image_dir = 'D:/ShailyDL/DeepL/final_split_dataset2/test/images'
label_dir = 'D:/ShailyDL/DeepL/final_split_dataset2/test/labels'

# Regex pattern to match filenames like *_C39.jpg (for crack)
pattern = re.compile(r".*_C\d+.*\.(?:jpg|jpeg|png|bmp|gif|webp)$", re.IGNORECASE)

# Get matched crack images
matched_images = [f for f in os.listdir(image_dir) if pattern.search(f.strip())]
matched_images.sort()

# Start renaming
for idx, image_name in enumerate(matched_images, start=1):
    name_base, ext = os.path.splitext(image_name)
    
    # Replace '_C' with '_D' to generate the label name for delamination if needed
    label_name = name_base + ".txt"
    label_path = os.path.join(label_dir, label_name)

    # New base name
    new_base = str(idx)

    # Paths for renaming image
    old_image_path = os.path.join(image_dir, image_name)
    new_image_path = os.path.join(image_dir, new_base + ext.lower())

    # Rename image
    os.rename(old_image_path, new_image_path)

    # Rename label if exists
    if os.path.exists(label_path):
        new_label_path = os.path.join(label_dir, new_base + ".txt")
        os.rename(label_path, new_label_path)
    else:
        # If label file doesn't exist as is, try '_D' variant
        delam_label_name = name_base.replace("_C", "_D") + ".txt"
        delam_label_path = os.path.join(label_dir, delam_label_name)

        if os.path.exists(delam_label_path):
            new_label_path = os.path.join(label_dir, new_base + ".txt")
            os.rename(delam_label_path, new_label_path)
        else:
            print(f"⚠️  Label not found for image: {image_name}")


In [3]:
!pip install pillow

   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ----------------------------------- ---- 2.4/2.7 MB 15.0 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 10.3 MB/s eta 0:00:00


In [1]:
import json
import os
import re
from PIL import Image

# Configuration
BASE_PATH = 'D:/ShailyDL/DeepL/final_split_dataset2'  # Update this path
BASE_PATH = 'D:/ShailyDL/DeepL/final_split_dataset2'

OUTPUT_DIR = 'coco_output'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# COCO dataset categories (must start from 1)
CATEGORIES = [
    {"id": 1, "name": "delamination"},
    {"id": 2, "name": "crack"}
]

# YOLO to COCO class ID mapping
CATEGORY_MAP = {
    0: 1,  # YOLO class 0 → COCO class 1 (delamination)
    1: 2   # YOLO class 1 → COCO class 2 (crack)
}

# Initialize COCO dataset structure
coco_output = {
    "info": {"description": "YOLO to COCO converted dataset"},
    "licenses": [],
    "categories": CATEGORIES,
    "images": [],
    "annotations": []
}

# Regex pattern to match valid image files
IMAGE_PATTERN = re.compile(r".*_[CD]\d+.*\.(?:jpg|jpeg|png|bmp|gif|webp)$", re.IGNORECASE)

def convert_yolo_to_coco():
    image_id = 0
    annotation_id = 0

    for split in ['train', 'val', 'test']:
        image_dir = os.path.join(BASE_PATH, split, 'images')
        label_dir = os.path.join(BASE_PATH, split, 'labels')

        if not os.path.exists(image_dir):
            print(f"Warning: {image_dir} does not exist. Skipping...")
            continue

        for filename in os.listdir(image_dir):
            if not IMAGE_PATTERN.search(filename):
                continue

            # Process image
            image_path = os.path.join(image_dir, filename)
            try:
                with Image.open(image_path) as img:
                    width, height = img.size
            except Exception as e:
                print(f"Error loading {image_path}: {str(e)}")
                continue

            # Add image metadata to COCO
            coco_output["images"].append({
                "id": image_id,
                "width": width,
                "height": height,
                "file_name": os.path.join(split, "images", filename).replace("\\", "/"),
                "split": split
            })

            # Process corresponding label file
            label_path = os.path.join(label_dir, os.path.splitext(filename)[0] + '.txt')
            if not os.path.exists(label_path):
                print(f"Warning: Missing label file for {filename}")
                continue

            with open(label_path, 'r') as f:
                for line_num, line in enumerate(f, 1):
                    parts = line.strip().split()
                    if len(parts) != 5:
                        print(f"Invalid annotation in {label_path}, line {line_num}: '{line.strip()}'")
                        continue

                    try:
                        cls, x_center, y_center, box_w, box_h = map(float, parts)
                        cls = int(cls)

                        # Convert YOLO to COCO bbox format [x_min, y_min, width, height]
                        x_min = round((x_center - box_w/2) * width)
                        y_min = round((y_center - box_h/2) * height)
                        width_px = round(box_w * width)
                        height_px = round(box_h * height)

                        # Ensure coordinates are within image bounds
                        x_min = max(0, x_min)
                        y_min = max(0, y_min)
                        width_px = min(width - x_min, width_px)
                        height_px = min(height - y_min, height_px)

                        if width_px <= 0 or height_px <= 0:
                            print(f"Invalid bbox in {label_path}, line {line_num}: {line.strip()}")
                            continue

                        coco_output["annotations"].append({
                            "id": annotation_id,
                            "image_id": image_id,
                            "category_id": CATEGORY_MAP[cls],
                            "bbox": [x_min, y_min, width_px, height_px],
                            "area": width_px * height_px,
                            "iscrowd": 0
                        })
                        annotation_id += 1

                    except (ValueError, KeyError) as e:
                        print(f"Error processing {label_path}, line {line_num}: {str(e)}")

            image_id += 1

    # Save COCO JSON
    output_path = os.path.join(OUTPUT_DIR, 'annotations.json')
    with open(output_path, 'w') as f:
        json.dump(coco_output, f, indent=4)

    print(f"\nConversion complete! Results saved to:\n{output_path}")
    print(f"Total images: {len(coco_output['images'])}")
    print(f"Total annotations: {len(coco_output['annotations'])}")

if __name__ == "__main__":
    convert_yolo_to_coco()


Conversion complete! Results saved to:
coco_output\annotations.json
Total images: 0
Total annotations: 0


In [2]:
import json
import os
from PIL import Image

BASE_PATH = 'D:/Deep_Learning/Dataset/MRCNN/'
OUTPUT_DIR = 'D:/Deep_Learning/Dataset/MRCNN/coco_output2'
os.makedirs(OUTPUT_DIR, exist_ok=True)

CATEGORIES = [{"id": 1, "name": "Delamination"}, {"id": 2, "name": "Crack"}, {"id": 3, "name": "Nodefect"}]
CATEGORY_MAP = {0: 1, 1: 2,  2: 3}  # YOLO to COCO class mapping

for split in ['train', 'val', 'test']:
    coco_data = {
        "info": {},
        "licenses": [],
        "categories": CATEGORIES,
        "images": [],
        "annotations": []
    }
    
    image_dir = os.path.join(BASE_PATH, split, 'images')
    label_dir = os.path.join(BASE_PATH, split, 'labels')
    
    image_id = 0
    annotation_id = 0
    
    for img_file in os.listdir(image_dir):
        if not img_file.lower().endswith(('.jpg', '.png', '.jpeg')):
            continue
        
        # Load image
        img_path = os.path.join(image_dir, img_file)
        with Image.open(img_path) as img:
            width, height = img.size
        
        # Add image info
        coco_data["images"].append({
            "id": image_id,
            "file_name": f"{split}/images/{img_file}",
            "width": width,
            "height": height
        })
        
        # Load annotations
        txt_file = os.path.splitext(img_file)[0] + '.txt'
        txt_path = os.path.join(label_dir, txt_file)
        
        if os.path.exists(txt_path):
            with open(txt_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) != 5:
                        continue
                    
                    cls, x_center, y_center, w, h = map(float, parts)
                    x_min = (x_center - w/2) * width
                    y_min = (y_center - h/2) * height
                    width_px = w * width
                    height_px = h * height
                    
                    coco_data["annotations"].append({
                        "id": annotation_id,
                        "image_id": image_id,
                        "category_id": CATEGORY_MAP[int(cls)],
                        "bbox": [x_min, y_min, width_px, height_px],
                        "area": width_px * height_px,
                        "iscrowd": 0
                    })
                    annotation_id += 1
        
        image_id += 1
    
    # Save per-split JSON
    with open(f'{OUTPUT_DIR}/annotations_{split}.json', 'w') as f:
        json.dump(coco_data, f, indent=2)

print("Conversion complete! Separate JSONs created for train/val/test.")

Conversion complete! Separate JSONs created for train/val/test.


In [12]:
!pip install matplotlib

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------  7.9/8.1 MB 44.5 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 38.4 MB/s eta 0:00:00
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 42.0 MB/s eta 0:00:00
Using cached pyparsing-3.2.3-py3-none-any.whl (111 kB)


In [12]:
import os
import json
import cv2
from pathlib import Path

# Path configuration
base_path = Path('D:/ShailyDL/DeepL/final_split_dataset_coco/test')
images_dir = base_path / 'images'
output_dir = base_path / 'visualized_bboxes'

# Try to find annotations in images folder
possible_annotation_names = [
    'annotations_test.json',
    'instances_test.json',
    'test.json',
    'annotations.json'
]

annotations_file = None
for name in possible_annotation_names:
    potential_path = images_dir / name
    if potential_path.exists():
        annotations_file = potential_path
        break

if not annotations_file:
    available_files = [f.name for f in images_dir.glob('*') if f.is_file()]
    raise FileNotFoundError(
        f"Could not find annotations file in {images_dir}.\n"
        f"Tried: {possible_annotation_names}\n"
        f"Files present: {available_files}"
    )

# Create output directory
output_dir.mkdir(exist_ok=True)

# Load COCO annotations
with open(annotations_file, 'r') as f:
    coco_data = json.load(f)

# Rest of the processing...
print(f"Found annotations file: {annotations_file}")
print(f"Found {len(coco_data['images'])} images in annotations")

# Create mapping from image_id to image info
image_info = {img['id']: img for img in coco_data['images']}
category_info = {cat['id']: cat['name'] for cat in coco_data['categories']}

colors = {
    1: (255, 0, 0),    # Red for delamination
    2: (0, 0, 255)      # Blue for crack
}

processed_images = 0

for ann in coco_data['annotations']:
    img_id = ann['image_id']
    img_info = image_info.get(img_id)
    
    if not img_info:
        print(f"Warning: No image info found for image_id {img_id}")
        continue

    img_filename = Path(img_info['file_name']).name
    img_path = images_dir / img_filename
    
    if not img_path.exists():
        print(f"Warning: Image not found at {img_path}")
        continue

    img = cv2.imread(str(img_path))
    if img is None:
        print(f"Warning: Could not load image {img_path}")
        continue
    
    # Draw bounding box
    bbox = ann['bbox']  # [x, y, width, height]
    category_id = ann['category_id']
    x, y, w, h = map(int, bbox)
    
    cv2.rectangle(img, (x, y), (x+w, y+h), colors[category_id], 2)
    
    # Add label text
    label = category_info.get(category_id, str(category_id))
    cv2.putText(img, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 
               0.5, colors.get(category_id, (0,255,0)), 2)
    
    # Save visualized image
    output_path = output_dir / img_filename
    cv2.imwrite(str(output_path), img)
    processed_images += 1

print(f"\nSuccessfully processed {processed_images} images")
print(f"Visualized images saved to: {output_dir}")

Found annotations file: D:\ShailyDL\DeepL\final_split_dataset_coco\test\images\annotations_test.json
Found 88 images in annotations

Successfully processed 102 images
Visualized images saved to: D:\ShailyDL\DeepL\final_split_dataset_coco\test\visualized_bboxes


In [1]:
import json
import os
from PIL import Image

BASE_PATH = 'D:/ShailyDL/DeepL/New_Detection_coco'
OUTPUT_DIR = 'coco_outputorg'
os.makedirs(OUTPUT_DIR, exist_ok=True)

CATEGORIES = [{"id": 1, "name": "delamination"}, {"id": 2, "name": "crack"}]
CATEGORY_MAP = {0: 1, 1: 2}  # YOLO to COCO class mapping

for split in ['train', 'val']:
    coco_data = {
        "info": {},
        "licenses": [],
        "categories": CATEGORIES,
        "images": [],
        "annotations": []
    }
    
    image_dir = os.path.join(BASE_PATH, split, 'images')
    label_dir = os.path.join(BASE_PATH, split, 'labels')
    
    image_id = 0
    annotation_id = 0
    
    for img_file in os.listdir(image_dir):
        if not img_file.lower().endswith(('.jpg', '.png', '.jpeg')):
            continue
        
        # Load image
        img_path = os.path.join(image_dir, img_file)
        with Image.open(img_path) as img:
            width, height = img.size
        
        # Add image info
        coco_data["images"].append({
            "id": image_id,
            "file_name": f"{split}/images/{img_file}",
            "width": width,
            "height": height
        })
        
        # Load annotations
        txt_file = os.path.splitext(img_file)[0] + '.txt'
        txt_path = os.path.join(label_dir, txt_file)
        
        if os.path.exists(txt_path):
            with open(txt_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) != 5:
                        continue
                    
                    cls, x_center, y_center, w, h = map(float, parts)
                    x_min = (x_center - w/2) * width
                    y_min = (y_center - h/2) * height
                    width_px = w * width
                    height_px = h * height
                    
                    coco_data["annotations"].append({
                        "id": annotation_id,
                        "image_id": image_id,
                        "category_id": CATEGORY_MAP[int(cls)],
                        "bbox": [x_min, y_min, width_px, height_px],
                        "area": width_px * height_px,
                        "iscrowd": 0
                    })
                    annotation_id += 1
        
        image_id += 1
    
    # Save per-split JSON
    with open(f'{OUTPUT_DIR}/annotations_{split}.json', 'w') as f:
        json.dump(coco_data, f, indent=2)

print("Conversion complete! Separate JSONs created for train/val")

Conversion complete! Separate JSONs created for train/val
